# Census datasets presence

*Goal:* demonstrate basic use of the `datasets_presence_matrix` array.

The presence matrix is a sparse array, indicating which features (var) were present in each dataset.  The array has dimensions [n_datasets, n_var], and is stored in the SOMA Measurement `varp` collection. The first dimension is indexed by the `soma_joinid` in the `census_datasets` dataframe. The second is indexed by the `soma_joinid` in the `var` dataframe of the measurement.

In [1]:
import numpy as np
from scipy import sparse
import cell_census

census = cell_census.open_soma()

# Grab the experiment containing human data, and the measurement therein with RNA
human = census["census_data"]["homo_sapiens"]
human_rna = human.ms["RNA"]

# The cell census-wide datasets
datasets_df = census["census_info"]["datasets"].read().concat().to_pandas()
datasets_df

,soma_joinid,collection_id,collection_name,collection_doi,dataset_id,dataset_title,dataset_h5ad_path,dataset_total_cell_count
0,0,03f821b4-87be-4ff4-b65a-b5fc00061da7,Local and systemic responses to SARS-CoV-2 inf...,10.1038/s41586-021-04345-x,edc8d3fe-153c-4e3d-8be0-2108d30f8d70,Airway,edc8d3fe-153c-4e3d-8be0-2108d30f8d70.h5ad,236977
1,1,03f821b4-87be-4ff4-b65a-b5fc00061da7,Local and systemic responses to SARS-CoV-2 inf...,10.1038/s41586-021-04345-x,2a498ace-872a-4935-984b-1afa70fd9886,PBMC,2a498ace-872a-4935-984b-1afa70fd9886.h5ad,422220
2,2,43d4bb39-21af-4d05-b973-4c1fed7b916c,Transcriptional Programming of Normal and Infl...,10.1016/j.celrep.2018.09.006,f512b8b6-369d-4a85-a695-116e0806857f,Skin,f512b8b6-369d-4a85-a695-116e0806857f.h5ad,68036
3,3,0434a9d4-85fd-4554-b8e3-cf6c582bb2fa,Acute COVID-19 cohort across a range of WHO ca...,10.1101/2020.11.20.20227355,fa8605cf-f27e-44af-ac2a-476bee4410d3,PBMCs,fa8605cf-f27e-44af-ac2a-476bee4410d3.h5ad,59506
4,4,a9254216-6cd8-4186-b32c-349363777584,Single-cell reconstruction of the early matern...,10.1038/s41586-018-0698-6,5bc42b88-bb76-4954-927b-8bb7369adc64,Pregnant Uterus (All),5bc42b88-bb76-4954-927b-8bb7369adc64.h5ad,70325
...,...,...,...,...,...,...,...,...
466,466,5d445965-6f1a-4b68-ba3a-b8f765155d3a,A molecular cell atlas of the human lung from ...,10.1038/s41586-020-2922-4,e04daea4-4412-45b5-989e-76a9be070a89,"Krasnow Lab Human Lung Cell Atlas, Smart-seq2",e04daea4-4412-45b5-989e-76a9be070a89.h5ad,9409
467,467,5d445965-6f1a-4b68-ba3a-b8f765155d3a,A molecular cell atlas of the human lung from ...,10.1038/s41586-020-2922-4,8c42cfd0-0b0a-46d5-910c-fc833d83c45e,"Krasnow Lab Human Lung Cell Atlas, 10X",8c42cfd0-0b0a-46d5-910c-fc833d83c45e.h5ad,65662
468,468,33d19f34-87f5-455b-8ca5-9023a2e5453d,Intra- and Inter-cellular Rewiring of the Huma...,10.1016/j.cell.2019.06.029,4dd00779-7f73-4f50-89bb-e2d3c6b71b18,Intra- and Inter-cellular Rewiring of the Huma...,4dd00779-7f73-4f50-89bb-e2d3c6b71b18.h5ad,34772
469,469,17481d16-ee44-49e5-bcf0-28c0780d8c4a,Single-Cell Sequencing of Developing Human Gut...,10.1016/j.devcel.2020.11.010,8e47ed12-c658-4252-b126-381df8d52a3d,Paediatric Human Gut (4-14y),8e47ed12-c658-4252-b126-381df8d52a3d.h5ad,22502


For convenience, read the entire presence matrix (for Homo sapiens) into a SciPy array. There is a convience API providing this capability, returning the matrix in a scipy.sparse.array:

In [2]:
presence_matrix = cell_census.get_presence_matrix(
    census, organism="Homo sapiens", measurement_name="RNA"
)
presence_matrix

<471x60664 sparse matrix of type '<class 'numpy.uint8'>'
	with 12385957 stored elements in Compressed Sparse Row format>

We also need the `var` dataframe, which is read into a Pandas DataFrame for convenient manipulation:

In [3]:
var_df = human_rna.var.read().concat().to_pandas()

## Is a feature present in a dataset?

*Goal:* test if a given feature is present in a given dataset.

**Important:** the presence matrix is indexed by soma_joinid, and is *NOT* positionally indexed.  In other words:
* the first dimension of the presence matrix is the dataset's `soma_joinid`, as stored in the `census_datasets` dataframe.
* the second dimension of the presence matrix is the feature's `soma_joinid`, as stored in the `var` dataframe.

In [4]:
var_joinid = var_df.loc[var_df.feature_id == "ENSG00000286096"].soma_joinid
dataset_joinid = datasets_df.loc[
    datasets_df.dataset_id == "97a17473-e2b1-4f31-a544-44a60773e2dd"
].soma_joinid
is_present = presence_matrix[dataset_joinid, var_joinid][0, 0]
print(f'Feature is {"present" if is_present else "not present"}.')

Feature is present.


## What datasets contain a feature?

*Goal:* look up all datasets that have a feature_id present.

In [5]:
# Grab the feature's soma_joinid from the var dataframe
var_joinid = var_df.loc[var_df.feature_id == "ENSG00000286096"].soma_joinid

# The presence matrix is indexed by the joinids of the dataset and var dataframes,
# so slice out the feature of interest by its joinid.
dataset_joinids = presence_matrix[:, var_joinid].tocoo().row

# From the datasets dataframe, slice out the datasets which have a joinid in the list
datasets_df.loc[datasets_df.soma_joinid.isin(dataset_joinids)]

,soma_joinid,collection_id,collection_name,collection_doi,dataset_id,dataset_title,dataset_h5ad_path,dataset_total_cell_count
283,283,e5f58829-1a66-40b5-a624-9046778e74f5,Tabula Sapiens,10.1126/science.abl4896,a68b64d8-aee3-4947-81b7-36b8fe5a44d2,Tabula Sapiens - Stromal,a68b64d8-aee3-4947-81b7-36b8fe5a44d2.h5ad,82478
284,284,e5f58829-1a66-40b5-a624-9046778e74f5,Tabula Sapiens,10.1126/science.abl4896,97a17473-e2b1-4f31-a544-44a60773e2dd,Tabula Sapiens - Epithelial,97a17473-e2b1-4f31-a544-44a60773e2dd.h5ad,104148
285,285,e5f58829-1a66-40b5-a624-9046778e74f5,Tabula Sapiens,10.1126/science.abl4896,c5d88abe-f23a-45fa-a534-788985e93dad,Tabula Sapiens - Immune,c5d88abe-f23a-45fa-a534-788985e93dad.h5ad,264824
286,286,e5f58829-1a66-40b5-a624-9046778e74f5,Tabula Sapiens,10.1126/science.abl4896,5a11f879-d1ef-458a-910c-9b0bdfca5ebf,Tabula Sapiens - Endothelial,5a11f879-d1ef-458a-910c-9b0bdfca5ebf.h5ad,31691
287,287,e5f58829-1a66-40b5-a624-9046778e74f5,Tabula Sapiens,10.1126/science.abl4896,53d208b0-2cfd-4366-9866-c3c6114081bc,Tabula Sapiens - All Cells,53d208b0-2cfd-4366-9866-c3c6114081bc.h5ad,483152
317,317,283d65eb-dd53-496d-adb7-7570c7caa443,Transcriptomic diversity of cell types across ...,10.1101/2022.10.12.511898,3a7f3ab4-a280-4b3b-b2c0-6dd05614a78c,Splatter,3a7f3ab4-a280-4b3b-b2c0-6dd05614a78c.h5ad,291833
320,320,283d65eb-dd53-496d-adb7-7570c7caa443,Transcriptomic diversity of cell types across ...,10.1101/2022.10.12.511898,bdb26abd-f4ba-4ea3-8862-c2340e7a4f55,CGE interneuron,bdb26abd-f4ba-4ea3-8862-c2340e7a4f55.h5ad,227671
330,330,283d65eb-dd53-496d-adb7-7570c7caa443,Transcriptomic diversity of cell types across ...,10.1101/2022.10.12.511898,5e5ab909-f73f-4b57-98a0-6d2c5662f6a4,Midbrain (M) - Inferior colliculus and nearby ...,5e5ab909-f73f-4b57-98a0-6d2c5662f6a4.h5ad,32306
353,353,283d65eb-dd53-496d-adb7-7570c7caa443,Transcriptomic diversity of cell types across ...,10.1101/2022.10.12.511898,35c8a04c-8639-4d15-8228-765d8d93fc96,Hypothalamus (HTH) - supraoptic region of HTH ...,35c8a04c-8639-4d15-8228-765d8d93fc96.h5ad,16753
358,358,283d65eb-dd53-496d-adb7-7570c7caa443,Transcriptomic diversity of cell types across ...,10.1101/2022.10.12.511898,3f56901c-dd4a-47d6-b60b-7b0c0111cfb2,Head of hippocampus (HiH) - CA1-3,3f56901c-dd4a-47d6-b60b-7b0c0111cfb2.h5ad,37911


## What features are in a dataset?

*Goal:* lookup the features present in a given dataset.

This example also demonstrates the ability to do the query on multiple datasets.

In [6]:
# Slice the dataset(s) of interest, and get the joinid(s)
dataset_joinids = datasets_df.loc[
    datasets_df.collection_id == "17481d16-ee44-49e5-bcf0-28c0780d8c4a"
].soma_joinid

# Slice the presence matrix by the first dimension, i.e., by dataset
var_joinids = presence_matrix[dataset_joinids, :].tocoo().col

# From the feature (var) dataframe, slice out features which have a joinid in the list.
var_df.loc[var_df.soma_joinid.isin(var_joinids)]

,soma_joinid,feature_id,feature_name,feature_length
0,0,ENSG00000121410,A1BG,3999
1,1,ENSG00000268895,A1BG-AS1,3374
2,2,ENSG00000148584,A1CF,9603
3,3,ENSG00000175899,A2M,6318
4,4,ENSG00000245105,A2M-AS1,2948
...,...,...,...,...
33438,33438,ENSG00000281741,CH17-118O6.6,1088
33439,33439,ENSG00000281880,PAUPAR,8565
33440,33440,ENSG00000232234,RP11-203H2.1,4624
33442,33442,ENSG00000255920,CCND2-AS1,4397
